In [1]:
# Cell 1: Import required libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully")
print(f"✓ TensorFlow version: {tf.__version__}")

✓ All libraries imported successfully
✓ TensorFlow version: 2.19.0


In [3]:
# Cell 2: Load the transformed datasets
print("="*50)
print("LOADING CNN-LSTM READY DATASET")
print("="*50)

# Load the datasets
df_transformed = pd.read_csv('cnn_lstm_ready_dataset.csv')
target_data = pd.read_csv('target_data_for_sequences.csv')

print(f"✓ Transformed data shape: {df_transformed.shape}")
print(f"✓ Target data shape: {target_data.shape}")
print(f"✓ Transformed data columns: {len(df_transformed.columns)} features")
print(f"✓ Target data columns: {len(target_data.columns)} target variables")

# Display first few rows
print(f"\nFirst 3 rows of transformed data:")
print(df_transformed.head(3))

LOADING CNN-LSTM READY DATASET
✓ Transformed data shape: (1458, 65)
✓ Target data shape: (1458, 4)
✓ Transformed data columns: 65 features
✓ Target data columns: 4 target variables

First 3 rows of transformed data:
       Year  CheckTotal   is_zero  IsRamadan     IsEid  IsPreRamadan  \
0 -0.575766    0.015624 -0.083103  -0.645685 -0.184506     -0.307562   
1 -0.575766    2.072745 -0.083103  -0.645685 -0.184506     -0.307562   
2 -0.575766   -0.155666 -0.083103  -0.645685 -0.184506     -0.307562   

   IsPostRamadan  IsLast10Ramadan     IsDSF  IsSummerEvent  ...  \
0      -0.307562        -0.207168 -0.261021      -0.389434  ...   
1      -0.307562        -0.207168 -0.261021      -0.389434  ...   
2      -0.307562        -0.207168 -0.261021      -0.389434  ...   

   Event_Ramadan-First10Days  Event_Ramadan-Last10Days  Event_Ramadan-Middle  \
0                  -0.207168                 -0.207168             -0.201706   
1                  -0.207168                 -0.207168            

In [ ]:
def create_sequences_for_cnn_lstm(df_transformed, target_data, sequence_length=30, forecast_horizon=7):
    """
    Create sequences for CNN-LSTM training from loaded CSV files
    """
    print("="*50)
    print("CREATING SEQUENCES FOR CNN-LSTM")
    print("="*50)
    
    # Parameters
    SEQ_LENGTH = sequence_length  # Look back 30 days
    FORECAST_HORIZON = forecast_horizon  # Predict next 7 days
    
    # Sort by date to ensure proper sequence order
    df_transformed_sorted = df_transformed.sort_values('Date').reset_index(drop=True)
    target_data_sorted = target_data.sort_values('Date').reset_index(drop=True)
    
    # Pivot target data to wide format
    target_pivot = target_data_sorted.pivot_table(
        index='Date', 
        columns=['RevenueCenterName', 'MealPeriod'], 
        values='CheckTotal', 
        fill_value=0
    ).reset_index()
    
    # Create column names for revenue streams
    target_pivot.columns = ['Date'] + [f"{col[0]}_{col[1]}" for col in target_pivot.columns[1:]]
    
    # Ensure same date range
    common_dates = set(df_transformed_sorted['Date']).intersection(set(target_pivot['Date']))
    df_transformed_sorted = df_transformed_sorted[df_transformed_sorted['Date'].isin(common_dates)].reset_index(drop=True)
    target_pivot = target_pivot[target_pivot['Date'].isin(common_dates)].reset_index(drop=True)
    
    # Remove Date column from features
    feature_columns = [col for col in df_transformed_sorted.columns if col != 'Date']
    features = df_transformed_sorted[feature_columns].values
    
    # Target columns (revenue targets)
    target_columns = [col for col in target_pivot.columns if col != 'Date']
    targets = target_pivot[target_columns].values
    
    print(f"✓ Feature shape: {features.shape}")
    print(f"✓ Target shape: {targets.shape}")
    print(f"✓ Number of feature columns: {len(feature_columns)}")
    print(f"✓ Number of target columns: {len(target_columns)}")
    print(f"✓ Target columns: {target_columns}")
    print(f"✓ Sequence length: {SEQ_LENGTH} days")
    print(f"✓ Forecast horizon: {FORECAST_HORIZON} days")
    
    # Create sequences
    X, y = [], []
    
    for i in range(SEQ_LENGTH, len(features) - FORECAST_HORIZON + 1):
        # Features: past 30 days
        X.append(features[i-SEQ_LENGTH:i])
        
        # Targets: next 7 days
        y.append(targets[i:i+FORECAST_HORIZON])
    
    X = np.array(X)
    y = np.array(y)
    
    print(f"✓ Final X shape: {X.shape}")  # (samples, 30, features)
    print(f"✓ Final y shape: {y.shape}")  # (samples, 7, revenue_targets)
    print(f"✓ Total sequences created: {len(X)}")
    
    return X, y, feature_columns, target_columns

In [7]:
# Cell 4: Corrected - Handle data without Date column in features
def clean_and_prepare_data_fixed(df_transformed, target_data):
    """
    Clean dataframes when features don't have Date column
    """
    print("="*50)
    print("CLEANING AND PREPARING DATA FOR CNN-LSTM")
    print("="*50)
    
    # Step 1: Check original data
    print("Original data info:")
    print(f"df_transformed shape: {df_transformed.shape}")
    print(f"df_transformed columns: {list(df_transformed.columns)}")
    print(f"target_data shape: {target_data.shape}")
    print(f"target_data columns: {list(target_data.columns)}")
    
    # Check if data is already aligned by length
    if len(df_transformed) == len(target_data):
        print("✓ Data lengths match - assuming already aligned by row index")
        
        # Step 2: Pivot target data from long to wide format
        print("\n🔄 Pivoting target data to wide format...")
        
        # Add row index to help with pivoting
        target_with_index = target_data.copy()
        target_with_index['row_index'] = target_with_index.index
        
        # Create a day identifier (since we know there are 3 meal periods per day)
        target_with_index['day_id'] = target_with_index['row_index'] // 3
        
        target_pivot = target_with_index.pivot_table(
            index='day_id', 
            columns='MealPeriod', 
            values='CheckTotal', 
            fill_value=0
        ).reset_index()
        
        print(f"✓ Pivoted target shape: {target_pivot.shape}")
        print(f"✓ Pivoted target columns: {list(target_pivot.columns)}")
        
        # Step 3: Aggregate features to day level (average of 3 meal periods per day)
        print("\n📊 Aggregating features to day level...")
        
        # Add day_id to features
        df_features_with_day = df_transformed.copy()
        df_features_with_day['day_id'] = df_features_with_day.index // 3
        
        # Aggregate features by day (mean of the 3 meal periods)
        df_features_daily = df_features_with_day.groupby('day_id').mean().reset_index()
        df_features_daily = df_features_daily.drop('day_id', axis=1)
        
        print(f"✓ Aggregated features shape: {df_features_daily.shape}")
        
        # Step 4: Align the data
        target_values = target_pivot.drop('day_id', axis=1)
        
        # Ensure same number of rows
        min_rows = min(len(df_features_daily), len(target_values))
        df_features_final = df_features_daily.iloc[:min_rows]
        target_values_final = target_values.iloc[:min_rows]
        
        print(f"✓ Final aligned shapes:")
        print(f"Features: {df_features_final.shape}")
        print(f"Targets: {target_values_final.shape}")
        
    else:
        raise ValueError(f"Data length mismatch: features={len(df_transformed)}, targets={len(target_data)}")
    
    # Step 5: Clean data types and handle missing values
    print("\n🧹 Cleaning data types...")
    
    # Features: ensure all numeric
    df_features_clean = df_features_final.select_dtypes(include=[np.number])
    df_features_clean = df_features_clean.fillna(0).astype(np.float32)
    
    # Targets: ensure all numeric
    df_targets_clean = target_values_final.fillna(0).astype(np.float32)
    
    print(f"✅ Final cleaned data:")
    print(f"Features shape: {df_features_clean.shape}")
    print(f"Targets shape: {df_targets_clean.shape}")
    print(f"Target columns: {list(df_targets_clean.columns)}")
    print(f"Data lengths match: {len(df_features_clean) == len(df_targets_clean)}")
    
    return df_features_clean, df_targets_clean

def create_sequences_for_cnn_lstm_corrected(df_features, df_targets, sequence_length=30, forecast_horizon=7):
    """
    Create sequences from properly aligned and cleaned data
    """
    print("\n" + "="*50)
    print("CREATING SEQUENCES FOR CNN-LSTM")
    print("="*50)
    
    # Parameters
    SEQ_LENGTH = sequence_length
    FORECAST_HORIZON = forecast_horizon
    
    # Convert to arrays
    features = df_features.values
    targets = df_targets.values
    feature_columns = df_features.columns.tolist()
    target_columns = df_targets.columns.tolist()
    
    print(f"✓ Feature shape: {features.shape}")
    print(f"✓ Target shape: {targets.shape}")
    print(f"✓ Sequence length: {SEQ_LENGTH} days")
    print(f"✓ Forecast horizon: {FORECAST_HORIZON} days")
    print(f"✓ Target columns: {target_columns}")
    
    # Verify we have enough data
    min_data_needed = SEQ_LENGTH + FORECAST_HORIZON
    if len(features) < min_data_needed:
        raise ValueError(f"Not enough data. Need at least {min_data_needed} rows, got {len(features)}")
    
    # Create sequences
    X, y = [], []
    
    for i in range(SEQ_LENGTH, len(features) - FORECAST_HORIZON + 1):
        # Features: past SEQ_LENGTH days
        X.append(features[i-SEQ_LENGTH:i])
        
        # Targets: next FORECAST_HORIZON days
        y.append(targets[i:i+FORECAST_HORIZON])
    
    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    
    print(f"✓ Final X shape: {X.shape}")  # (samples, sequence_length, features)
    print(f"✓ Final y shape: {y.shape}")  # (samples, forecast_horizon, revenue_streams)
    print(f"✓ X dtype: {X.dtype}")
    print(f"✓ y dtype: {y.dtype}")
    print(f"✓ Total sequences created: {len(X)}")
    
    # Show example of what each dimension means
    print(f"\n📊 Shape interpretation:")
    print(f"X: ({X.shape[0]} sequences, {X.shape[1]} days history, {X.shape[2]} features)")
    print(f"y: ({y.shape[0]} sequences, {y.shape[1]} days forecast, {y.shape[2]} revenue streams)")
    
    return X, y, feature_columns, target_columns

# Execute the corrected pipeline
try:
    # Step 1: Clean and prepare data without Date column dependency
    df_features_clean, df_targets_clean = clean_and_prepare_data_fixed(df_transformed, target_data)
    
    # Step 2: Create sequences
    X, y, feature_cols, target_cols = create_sequences_for_cnn_lstm_corrected(
        df_features_clean, df_targets_clean
    )
    
    print(f"\n🎉 SUCCESS! Sequences created successfully!")
    print(f"✓ Input sequences (X): {X.shape}")
    print(f"✓ Output sequences (y): {y.shape}")
    print(f"✓ Feature columns: {len(feature_cols)}")
    print(f"✓ Target columns: {target_cols}")
    print(f"✓ Data types: X={X.dtype}, y={y.dtype}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

CLEANING AND PREPARING DATA FOR CNN-LSTM
Original data info:
df_transformed shape: (1458, 65)
df_transformed columns: ['Year', 'CheckTotal', 'is_zero', 'IsRamadan', 'IsEid', 'IsPreRamadan', 'IsPostRamadan', 'IsLast10Ramadan', 'IsDSF', 'IsSummerEvent', 'IsNationalDay', 'IsNewYear', 'IsMarathon', 'IsGITEX', 'IsAirshow', 'IsFoodFestival', 'IsPreEvent', 'IsPostEvent', 'Month_sin', 'Month_cos', 'DayOfWeek_sin', 'DayOfWeek_cos', 'Meal_Breakfast', 'Meal_Dinner', 'Meal_Lunch', 'Event_Dubai-Airshow', 'Event_Dubai-Food-Festival', 'Event_Dubai-Marathon', 'Event_Dubai-Shopping-Festival', 'Event_Dubai-Summer-Surprises', 'Event_Eid-Adha', 'Event_Flag-Day', 'Event_GITEX-Technology-Week', 'Event_New-Year-Celebrations', 'Event_Normal', 'Event_Post-Dubai-Airshow', 'Event_Post-Dubai-Marathon', 'Event_Post-Eid-Adha', 'Event_Post-Flag-Day', 'Event_Post-GITEX-Technology-Week', 'Event_Post-New-Year-Celebrations', 'Event_Post-Ramadan-Recovery', 'Event_Post-Ramadan-Week1', 'Event_Post-Summer-Event', 'Event_Pre

In [17]:
print(X)

[[[-0.5757663   0.6442341  -0.08310281 ... -0.3115533  -2.6150928
    4.957716  ]
  [-0.5757663   0.20041224 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  [-0.5757663   0.10293791 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  ...
  [-0.5757663   0.01185533 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  [-0.5757663   0.11598377 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  [-0.5757663  -0.17047678 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]]

 [[-0.5757663   0.20041224 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  [-0.5757663   0.10293791 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  [-0.5757663   0.44010323 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  ...
  [-0.5757663   0.11598377 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  [-0.5757663  -0.17047678 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  [-0.5757663   0.15700552 -0.08310281 ... -0.3115533   0.38239563
   -0.2

In [11]:
# Cell 5: Train-Test Split with clean data
print("="*30)
print("TRAIN-TEST SPLIT")
print("="*30)

# Time-based split (80% train, 20% test)
split_ratio = 0.8
split_index = int(len(X) * split_ratio)

X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

print(f"✓ Training sequences: {X_train.shape[0]}")
print(f"✓ Testing sequences: {X_test.shape[0]}")
print(f"✓ Input shape per sample: {X_train.shape[1:]}")
print(f"✓ Output shape per sample: {y_train.shape[1:]}")

# Verify data types
print(f"✓ X_train dtype: {X_train.dtype}")
print(f"✓ y_train dtype: {y_train.dtype}")
print(f"✓ X_test dtype: {X_test.dtype}")
print(f"✓ y_test dtype: {y_test.dtype}")

# Check for any problematic values
print(f"\n✓ Data quality check:")
print(f"X_train NaN count: {np.isnan(X_train).sum()}")
print(f"y_train NaN count: {np.isnan(y_train).sum()}")
print(f"X_train Inf count: {np.isinf(X_train).sum()}")
print(f"y_train Inf count: {np.isinf(y_train).sum()}")

print(f"\n✅ Data is ready for training!")

TRAIN-TEST SPLIT
✓ Training sequences: 360
✓ Testing sequences: 90
✓ Input shape per sample: (30, 65)
✓ Output shape per sample: (7, 3)
✓ X_train dtype: float32
✓ y_train dtype: float32
✓ X_test dtype: float32
✓ y_test dtype: float32

✓ Data quality check:
X_train NaN count: 0
y_train NaN count: 0
X_train Inf count: 0
y_train Inf count: 0

✅ Data is ready for training!


In [12]:
# Cell 6: Define CNN-LSTM model architecture
def build_cnn_lstm_model(input_shape, output_shape):
    """
    Build CNN-LSTM hybrid model for hotel revenue forecasting
    """
    print(f"✓ Building model with input shape: {input_shape}")
    print(f"✓ Output shape: {output_shape}")
    
    model = Sequential([
        # CNN layers for feature extraction
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape, name='conv1d_1'),
        Conv1D(filters=64, kernel_size=3, activation='relu', name='conv1d_2'),
        MaxPooling1D(pool_size=2, name='maxpool_1'),
        Dropout(0.2, name='dropout_1'),
        
        # More CNN layers
        Conv1D(filters=32, kernel_size=3, activation='relu', name='conv1d_3'),
        MaxPooling1D(pool_size=2, name='maxpool_2'),
        Dropout(0.2, name='dropout_2'),
        
        # LSTM layers for temporal patterns
        LSTM(100, return_sequences=True, name='lstm_1'),
        Dropout(0.3, name='dropout_3'),
        LSTM(50, return_sequences=False, name='lstm_2'),
        Dropout(0.3, name='dropout_4'),
        
        # Dense layers for final prediction
        Dense(100, activation='relu', name='dense_1'),
        Dropout(0.2, name='dropout_5'),
        Dense(np.prod(output_shape), activation='linear', name='dense_output'),
    ])
    
    # Reshape output to (forecast_days, revenue_streams)
    model.add(tf.keras.layers.Reshape(output_shape, name='reshape_output'))
    
    return model

print("✓ Model building function defined")

✓ Model building function defined


In [13]:
# Cell 7: Build and compile the model
print("="*30)
print("BUILDING MODEL")
print("="*30)

# Define input and output shapes
input_shape = (X_train.shape[1], X_train.shape[2])  # (30, features)
output_shape = (y_train.shape[1], y_train.shape[2])  # (7, revenue_streams)

# Build model
model = build_cnn_lstm_model(input_shape, output_shape)

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

print("\n" + "="*30)
print("MODEL ARCHITECTURE")
print("="*30)
model.summary()

# Count parameters
total_params = model.count_params()
print(f"\n✓ Total parameters: {total_params:,}")

BUILDING MODEL
✓ Building model with input shape: (30, 65)
✓ Output shape: (7, 3)

MODEL ARCHITECTURE


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 28, 64)         │        12,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 26, 64)         │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_1 (MaxPooling1D)        │ (None, 13, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 13, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 11, 32)         │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_2 (MaxPooling1D)        │ (None, 5, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 5, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 5, 100)         │        53,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 5, 100)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 50)             │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │         5,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_output (Dense)            │ (None, 21)             │         2,121 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_output (Reshape)        │ (None, 7, 3)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 121,693 (475.36 KB)

 Trainable params: 121,693 (475.36 KB)

 Non-trainable params: 0 (0.00 B)


✓ Total parameters: 121,693


In [14]:
# Cell 8: Setup training callbacks
print("="*30)
print("TRAINING SETUP")
print("="*30)

# Define callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss', 
        patience=15, 
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.5, 
        patience=5, 
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        'best_cnn_lstm_model.h5', 
        save_best_only=True, 
        monitor='val_loss',
        verbose=1
    )
]

# Training parameters
BATCH_SIZE = 32
EPOCHS = 100

print("✓ Callbacks configured:")
print("  - Early stopping (patience=15)")
print("  - Learning rate reduction (factor=0.5, patience=5)")
print("  - Model checkpoint (best_cnn_lstm_model.h5)")
print(f"✓ Batch size: {BATCH_SIZE}")
print(f"✓ Max epochs: {EPOCHS}")

TRAINING SETUP
✓ Callbacks configured:
  - Early stopping (patience=15)
  - Learning rate reduction (factor=0.5, patience=5)
  - Model checkpoint (best_cnn_lstm_model.h5)
✓ Batch size: 32
✓ Max epochs: 100


In [15]:
# Cell 9 Alternative: Comprehensive data cleaning and training
import tensorflow as tf

print("="*40)
print("COMPREHENSIVE DATA PREPARATION")
print("="*40)

def clean_and_prepare_data(X_train, y_train, X_test, y_test):
    """
    Comprehensive data cleaning for CNN-LSTM training
    """
    print("🔧 Cleaning and preparing data...")
    
    # Convert to numpy arrays if not already
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    
    # Check for object dtype issues
    if X_train.dtype == 'object':
        print("⚠️  X_train has object dtype - converting...")
        X_train = X_train.astype(np.float64)
    
    if y_train.dtype == 'object':
        print("⚠️  y_train has object dtype - converting...")
        y_train = y_train.astype(np.float64)
    
    if X_test.dtype == 'object':
        print("⚠️  X_test has object dtype - converting...")
        X_test = X_test.astype(np.float64)
    
    if y_test.dtype == 'object':
        print("⚠️  y_test has object dtype - converting...")
        y_test = y_test.astype(np.float64)
    
    # Handle NaN and infinite values
    print("🧹 Handling NaN and infinite values...")
    X_train = np.nan_to_num(X_train, nan=0.0, posinf=1e6, neginf=-1e6)
    y_train = np.nan_to_num(y_train, nan=0.0, posinf=1e6, neginf=-1e6)
    X_test = np.nan_to_num(X_test, nan=0.0, posinf=1e6, neginf=-1e6)
    y_test = np.nan_to_num(y_test, nan=0.0, posinf=1e6, neginf=-1e6)
    
    # Convert to float32 (TensorFlow's preferred type)
    X_train = X_train.astype(np.float32)
    y_train = y_train.astype(np.float32)
    X_test = X_test.astype(np.float32)
    y_test = y_test.astype(np.float32)
    
    # Final verification
    print(f"✓ Final data types:")
    print(f"  X_train: {X_train.dtype}, shape: {X_train.shape}")
    print(f"  y_train: {y_train.dtype}, shape: {y_train.shape}")
    print(f"  X_test: {X_test.dtype}, shape: {X_test.shape}")
    print(f"  y_test: {y_test.dtype}, shape: {y_test.shape}")
    
    # Check data ranges
    print(f"✓ Data ranges:")
    print(f"  X_train: [{X_train.min():.3f}, {X_train.max():.3f}]")
    print(f"  y_train: [{y_train.min():.3f}, {y_train.max():.3f}]")
    
    return X_train, y_train, X_test, y_test

# Clean the data
X_train_clean, y_train_clean, X_test_clean, y_test_clean = clean_and_prepare_data(
    X_train, y_train, X_test, y_test
)

print("\n" + "="*30)
print("STARTING TRAINING")
print("="*30)

# Train with cleaned data
try:
    history = model.fit(
        X_train_clean, y_train_clean,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_test_clean, y_test_clean),
        callbacks=callbacks,
        verbose=1
    )
    
    print("\n✅ Training completed successfully!")
    
    # Update variables for next cells
    X_train, y_train = X_train_clean, y_train_clean
    X_test, y_test = X_test_clean, y_test_clean
    
except Exception as e:
    print(f"❌ Training still failed: {e}")
    print("\n🔍 Additional debugging:")
    
    # More detailed debugging
    print(f"X_train unique dtypes: {set(str(x.dtype) for x in X_train.flatten()[:100])}")
    print(f"Sample X_train values: {X_train[0, 0, :10]}")
    print(f"Sample y_train values: {y_train[0, 0, :10]}")
    
    # Check if data contains any strings
    sample_x = X_train[0, 0, :]
    print(f"Sample X contains strings: {any(isinstance(x, str) for x in sample_x.flatten())}")

COMPREHENSIVE DATA PREPARATION
🔧 Cleaning and preparing data...
🧹 Handling NaN and infinite values...
✓ Final data types:
  X_train: float32, shape: (360, 30, 65)
  y_train: float32, shape: (360, 7, 3)
  X_test: float32, shape: (90, 30, 65)
  y_test: float32, shape: (90, 7, 3)
✓ Data ranges:
  X_train: [-2.615, 22.023]
  y_train: [5.000, 10052.500]

STARTING TRAINING
Epoch 1/100
11/12 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 3742371.5000 - mae: 1385.3073
Epoch 1: val_loss improved from inf to 7599098.50000, saving model to best_cnn_lstm_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 83ms/step - loss: 3687530.5000 - mae: 1378.0475 - val_loss: 7599098.5000 - val_mae: 2076.6685 - learning_rate: 0.0010
Epoch 2/100
10/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3415606.2500 - mae: 1343.6519
Epoch 2: val_loss improved from 7599098.50000 to 7588302.00000, saving model to best_cnn_lstm_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 3409511.2500 - mae: 1342.3674 - val_loss: 7588302.0000 - val_mae: 2074.4189 - learning_rate: 0.0010
Epoch 3/100
10/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3554489.0000 - mae: 1366.9968
Epoch 3: val_loss improved from 7588302.00000 to 7570808.00000, saving model to best_cnn_lstm_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 3514479.2500 - mae: 1359.7239 - val_loss: 7570808.0000 - val_mae: 2070.5779 - learning_rate: 0.0010
Epoch 4/100
 7/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3461917.7500 - mae: 1350.1952 
Epoch 4: val_loss improved from 7570808.00000 to 7545618.00000, saving model to best_cnn_lstm_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 3434490.7500 - mae: 1343.0486 - val_loss: 7545618.0000 - val_mae: 2064.8682 - learning_rate: 0.0010
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 3151091.0000 - mae: 1294.2996
Epoch 5: val_loss improved from 7545618.00000 to 7511493.50000, saving model to best_cnn_lstm_model.h5


...

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 939209.1875 - mae: 620.8265 - val_loss: 2830357.5000 - val_mae: 1160.9896 - learning_rate: 0.0010
Epoch 75/100
 7/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1053881.6250 - mae: 635.6300 
Epoch 75: val_loss improved from 2830357.50000 to 2825306.75000, saving model to best_cnn_lstm_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 990843.4375 - mae: 619.0139 - val_loss: 2825306.7500 - val_mae: 1160.3972 - learning_rate: 0.0010
Epoch 76/100
 6/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 912400.8750 - mae: 612.6064
Epoch 76: val_loss improved from 2825306.75000 to 2814419.00000, saving model to best_cnn_lstm_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 913937.6250 - mae: 608.2424 - val_loss: 2814419.0000 - val_mae: 1158.5679 - learning_rate: 0.0010
Epoch 77/100
10/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 845614.6875 - mae: 592.6949
Epoch 77: val_loss improved from 2814419.00000 to 2813284.00000, saving model to best_cnn_lstm_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 869222.0000 - mae: 598.5742 - val_loss: 2813284.0000 - val_mae: 1158.4034 - learning_rate: 0.0010
Epoch 78/100
 7/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 890656.6250 - mae: 595.2648 
Epoch 78: val_loss did not improve from 2813284.00000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 903234.9375 - mae: 599.7598 - val_loss: 2818944.2500 - val_mae: 1159.3268 - learning_rate: 0.0010
Epoch 79/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 998480.7500 - mae: 614.8575 
Epoch 79: val_loss did not improve from 2813284.00000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 992196.6875 - mae: 613.9543 - val_loss: 2817175.7500 - val_mae: 1159.0815 - learning_rate: 0.0010
Epoch 80/100
 8/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 931143.9375 - mae: 609.6587 
Epoch 80: val_loss did not improve from 2813284.00000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 924395.4375 - mae: 607.1669 - val_loss: 2817817.5000 - val_mae: 1159.2218 - learning

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 861456.4375 - mae: 589.8607 - val_loss: 2813086.7500 - val_mae: 1158.6277 - learning_rate: 5.0000e-04
Epoch 84/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 868592.8750 - mae: 601.8333
Epoch 84: val_loss did not improve from 2813086.75000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 872780.0000 - mae: 602.4469 - val_loss: 2813775.7500 - val_mae: 1158.7277 - learning_rate: 5.0000e-04
Epoch 85/100
 8/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 919723.0000 - mae: 606.6731 
Epoch 85: val_loss did not improve from 2813086.75000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 931386.6250 - mae: 608.7726 - val_loss: 2814635.2500 - val_mae: 1158.8400 - learning_rate: 5.0000e-04
Epoch 86/100
11/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 845282.6250 - mae: 593.6076
Epoch 86: val_loss did not improve from 2813086.75000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 858293.8125 - mae: 596.2725 - val_loss: 2817462.0000 - val_mae: 1159.2678

In [16]:
# Cell 10: Fixed Model Evaluation with shape checking
print("="*30)
print("COMPREHENSIVE MODEL EVALUATION")
print("="*30)

# Make predictions on test set
y_pred = model.predict(X_test, verbose=0)

# Debug shapes first
print(f"🔍 Shape Debugging:")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")
print(f"y_pred shape: {y_pred.shape}")

# Check if shapes match
if y_test.shape != y_pred.shape:
    print(f"⚠️  Shape mismatch detected!")
    print(f"y_test: {y_test.shape} vs y_pred: {y_pred.shape}")
    
    # Try to fix shape mismatch
    if len(y_pred.shape) == 2 and len(y_test.shape) == 3:
        # If y_pred is 2D, reshape to 3D
        expected_shape = (y_pred.shape[0], y_test.shape[1], y_test.shape[2])
        if y_pred.shape[1] == (y_test.shape[1] * y_test.shape[2]):
            y_pred = y_pred.reshape(expected_shape)
            print(f"✅ Reshaped y_pred to: {y_pred.shape}")
        else:
            print(f"❌ Cannot reshape y_pred automatically")
            print(f"Expected elements: {y_test.shape[1] * y_test.shape[2]}, got: {y_pred.shape[1]}")
    
    elif len(y_pred.shape) == 3 and len(y_test.shape) == 3:
        if y_pred.shape[0] != y_test.shape[0]:
            # Take only matching samples
            min_samples = min(y_pred.shape[0], y_test.shape[0])
            y_pred = y_pred[:min_samples]
            y_test = y_test[:min_samples]
            print(f"✅ Trimmed to {min_samples} samples")

# Verify shapes again
print(f"\n✅ Final shapes:")
print(f"y_test: {y_test.shape}")
print(f"y_pred: {y_pred.shape}")

# Calculate overall metrics
try:
    # Flatten arrays for overall metrics
    y_test_flat = y_test.reshape(-1)
    y_pred_flat = y_pred.reshape(-1)
    
    print(f"\nFlattened shapes: y_test={y_test_flat.shape}, y_pred={y_pred_flat.shape}")
    
    mae = mean_absolute_error(y_test_flat, y_pred_flat)
    mse = mean_squared_error(y_test_flat, y_pred_flat)
    rmse = np.sqrt(mse)
    
    # Calculate MAPE (avoiding division by zero)
    mape = np.mean(np.abs((y_test_flat - y_pred_flat) / (np.abs(y_test_flat) + 1e-8))) * 100
    
    print(f"\n✅ Overall Test Metrics:")
    print(f"  MAE: {mae:.2f}")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  MSE: {mse:.2f}")
    print(f"  MAPE: {mape:.2f}%")
    
    # Calculate metrics by forecast day
    print(f"\n✅ Performance by Forecast Day:")
    for day in range(y_test.shape[1]):  # Should be 7 days
        day_mae = mean_absolute_error(y_test[:, day, :].reshape(-1), y_pred[:, day, :].reshape(-1))
        print(f"  Day {day+1}: MAE = {day_mae:.2f}")
    
    # Calculate metrics by revenue stream
    print(f"\n✅ Performance by Revenue Stream:")
    revenue_stream_metrics = []
    for stream in range(y_test.shape[2]):  # Should be 27 streams
        stream_mae = mean_absolute_error(y_test[:, :, stream].reshape(-1), y_pred[:, :, stream].reshape(-1))
        revenue_stream_metrics.append(stream_mae)
        if stream < 10:  # Show first 10 streams to avoid clutter
            print(f"  Stream {stream+1}: MAE = {stream_mae:.2f}")
    
    print(f"\n✅ Revenue Stream Summary:")
    print(f"  Best performing stream: MAE = {min(revenue_stream_metrics):.2f}")
    print(f"  Worst performing stream: MAE = {max(revenue_stream_metrics):.2f}")
    print(f"  Average across streams: MAE = {np.mean(revenue_stream_metrics):.2f}")
    print(f"  Standard deviation: MAE = {np.std(revenue_stream_metrics):.2f}")
    
    # Sample predictions vs actual
    print(f"\n✅ Sample Predictions (First sequence, First 5 streams):")
    print("Day | Stream1_Actual | Stream1_Pred | Stream2_Actual | Stream2_Pred | ...")
    for day in range(min(7, y_test.shape[1])):
        actual_vals = y_test[0, day, :5]  # First 5 streams
        pred_vals = y_pred[0, day, :5]
        print(f"{day+1:2d}  | {actual_vals[0]:8.1f}      | {pred_vals[0]:8.1f}     | {actual_vals[1]:8.1f}      | {pred_vals[1]:8.1f}     | ...")
    
    print(f"\n✅ Model evaluation complete!")
    
except Exception as e:
    print(f"❌ Error during evaluation: {e}")
    print(f"Shapes: y_test={y_test.shape}, y_pred={y_pred.shape}")
    
    # Fallback: simple metrics calculation
    print(f"\n🔧 Attempting simplified evaluation...")
    try:
        # Calculate MAE for each sample individually then average
        sample_maes = []
        for i in range(min(len(y_test), len(y_pred))):
            sample_mae = np.mean(np.abs(y_test[i] - y_pred[i]))
            sample_maes.append(sample_mae)
        
        overall_mae = np.mean(sample_maes)
        print(f"✅ Simplified MAE: {overall_mae:.2f}")
        print(f"✅ MAE std: {np.std(sample_maes):.2f}")
        
    except Exception as e2:
        print(f"❌ Simplified evaluation also failed: {e2}")

COMPREHENSIVE MODEL EVALUATION
🔍 Shape Debugging:
X_test shape: (90, 30, 65)
y_test shape: (90, 7, 3)
y_pred shape: (90, 7, 3)

✅ Final shapes:
y_test: (90, 7, 3)
y_pred: (90, 7, 3)

Flattened shapes: y_test=(1890,), y_pred=(1890,)

✅ Overall Test Metrics:
  MAE: 1158.63
  RMSE: 1677.23
  MSE: 2813087.25
  MAPE: 85.00%

✅ Performance by Forecast Day:
  Day 1: MAE = 1218.02
  Day 2: MAE = 1196.12
  Day 3: MAE = 1175.83
  Day 4: MAE = 1155.93
  Day 5: MAE = 1132.07
  Day 6: MAE = 1121.33
  Day 7: MAE = 1111.09

✅ Performance by Revenue Stream:
  Stream 1: MAE = 1010.31
  Stream 2: MAE = 1749.13
  Stream 3: MAE = 716.44

✅ Revenue Stream Summary:
  Best performing stream: MAE = 716.44
  Worst performing stream: MAE = 1749.13
  Average across streams: MAE = 1158.63
  Standard deviation: MAE = 434.44

✅ Sample Predictions (First sequence, First 5 streams):
Day | Stream1_Actual | Stream1_Pred | Stream2_Actual | Stream2_Pred | ...
 1  |   2466.0      |    801.4     |   6548.0      |   2464.6 